In [5]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from matplotlib import image
from matplotlib import pyplot as plt
import numpy as np; import sys

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

model = ResNet50(weights='imagenet')

In [7]:
banana = preprocess_input(np.expand_dims(image.img_to_array(image.load_img('banana.png', target_size=(224, 224))), axis=0))
hamburguer = preprocess_input(np.expand_dims(image.img_to_array(image.load_img('hamburguer.png', target_size=(224, 224))), axis=0))
bola = preprocess_input(np.expand_dims(image.img_to_array(image.load_img('bola.png', target_size=(224, 224))), axis=0))

In [8]:
preds = model.predict(banana)
print('Predicted:', decode_predictions(preds, top=3)[0])

preds = model.predict(hamburguer)
print('Predicted:', decode_predictions(preds, top=3)[0])

preds = model.predict(bola)
print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 1s 1s/step
Predicted: [('n07753592', 'banana', 0.99999464), ('n03532672', 'hook', 3.9854626e-06), ('n07747607', 'orange', 5.866019e-07)]
1/1 [==============================] - 0s 139ms/step
Predicted: [('n07697313', 'cheeseburger', 0.99954647), ('n07693725', 'bagel', 0.000313598), ('n07583066', 'guacamole', 0.0001260351)]
1/1 [==============================] - 0s 96ms/step
Predicted: [('n04037443', 'racer', 0.47095188), ('n04465501', 'tractor', 0.38897225), ('n04461696', 'tow_truck', 0.034972064)]
